#### Simple notebook to run model tests only

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train.parquet')

In [3]:
X_test['month'] = X_test.index.strftime('%B')
X_train["month"] = X_train.index.strftime('%B')

## Catboost

In [4]:
from catboost import Pool, CatBoostRegressor

#Create a pool of data
train_pool = Pool(X_train, y_train, cat_features=["location", "month"])
test_pool = Pool(X_test, cat_features=["location", "month"]) 

cat_feature = ["location", "month"]

#init model and fit it
catboost_model = CatBoostRegressor(cat_features=cat_feature)
catboost_model.fit(train_pool)

Learning rate set to 0.083785
0:	learn: 715.4942395	total: 230ms	remaining: 3m 49s
1:	learn: 668.8415646	total: 361ms	remaining: 3m
2:	learn: 626.5740437	total: 491ms	remaining: 2m 43s
3:	learn: 588.5596913	total: 685ms	remaining: 2m 50s
4:	learn: 554.5945296	total: 811ms	remaining: 2m 41s
5:	learn: 524.4299435	total: 918ms	remaining: 2m 32s
6:	learn: 496.9040892	total: 1.02s	remaining: 2m 24s
7:	learn: 471.8172691	total: 1.11s	remaining: 2m 18s
8:	learn: 450.6315178	total: 1.23s	remaining: 2m 15s
9:	learn: 430.9116951	total: 1.34s	remaining: 2m 12s
10:	learn: 413.6618005	total: 1.44s	remaining: 2m 9s
11:	learn: 398.8935962	total: 1.55s	remaining: 2m 7s
12:	learn: 385.1490574	total: 1.66s	remaining: 2m 5s
13:	learn: 372.9535399	total: 1.75s	remaining: 2m 3s
14:	learn: 362.1769068	total: 1.86s	remaining: 2m 2s
15:	learn: 352.9804360	total: 1.96s	remaining: 2m
16:	learn: 344.8142110	total: 2.05s	remaining: 1m 58s
17:	learn: 337.5011899	total: 2.15s	remaining: 1m 57s
18:	learn: 331.118802

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
ts = TimeSeriesSplit(n_splits = 10)
sklearn_cv_result = cross_val_score(catboost_model, X_train, y_train, cv=ts, scoring='neg_mean_absolute_error')

Learning rate set to 0.057363
0:	learn: 1025.7184973	total: 66ms	remaining: 1m 5s
1:	learn: 985.4868834	total: 129ms	remaining: 1m 4s
2:	learn: 946.5727352	total: 195ms	remaining: 1m 4s
3:	learn: 909.9935789	total: 284ms	remaining: 1m 10s
4:	learn: 876.6268535	total: 372ms	remaining: 1m 14s
5:	learn: 844.7490666	total: 453ms	remaining: 1m 14s
6:	learn: 816.9167513	total: 548ms	remaining: 1m 17s
7:	learn: 789.7007807	total: 622ms	remaining: 1m 17s
8:	learn: 764.1439938	total: 711ms	remaining: 1m 18s
9:	learn: 740.7559389	total: 787ms	remaining: 1m 17s
10:	learn: 720.4501525	total: 859ms	remaining: 1m 17s
11:	learn: 700.1998592	total: 933ms	remaining: 1m 16s
12:	learn: 682.7629267	total: 1.02s	remaining: 1m 17s
13:	learn: 665.0970387	total: 1.1s	remaining: 1m 17s
14:	learn: 650.2235891	total: 1.18s	remaining: 1m 17s
15:	learn: 635.0037568	total: 1.25s	remaining: 1m 17s
16:	learn: 621.8730837	total: 1.33s	remaining: 1m 16s
17:	learn: 609.2648048	total: 1.4s	remaining: 1m 16s
18:	learn: 59

In [8]:
np.mean(sklearn_cv_result)

-156.23845330093837

In [9]:
#make predictions
predictions = pd.DataFrame(catboost_model.predict(test_pool))
print(predictions)

             0
0    -1.374994
1    -1.596107
2    -4.662319
3    -3.671793
4    -8.379227
...        ...
8635  6.313554
8636  3.695525
8637  5.403859
8638  6.631637
8639  6.160030

[8640 rows x 1 columns]


In [25]:
#Local testing
from catboost import cv
cv_results = cv(train_pool, catboost_model.get_params(), fold_count=2, type="TimeSeries")

Training on fold [0/2]
0:	learn: 1312.8228543	test: 387.5876260	best: 387.5876260 (0)	total: 14.2ms	remaining: 14.2s
1:	learn: 1280.4187574	test: 368.4074392	best: 368.4074392 (1)	total: 26.3ms	remaining: 13.1s
2:	learn: 1248.8584366	test: 352.0701202	best: 352.0701202 (2)	total: 40.4ms	remaining: 13.4s
3:	learn: 1218.5941554	test: 338.8313140	best: 338.8313140 (3)	total: 52.1ms	remaining: 13s
4:	learn: 1189.6026450	test: 328.0818354	best: 328.0818354 (4)	total: 66.6ms	remaining: 13.3s
5:	learn: 1162.3631465	test: 320.2438678	best: 320.2438678 (5)	total: 83.1ms	remaining: 13.8s
6:	learn: 1135.8704857	test: 316.0048521	best: 316.0048521 (6)	total: 97.9ms	remaining: 13.9s
7:	learn: 1110.0830953	test: 313.1415179	best: 313.1415179 (7)	total: 115ms	remaining: 14.2s
8:	learn: 1084.8561608	test: 312.7655298	best: 312.7655298 (8)	total: 135ms	remaining: 14.8s
9:	learn: 1060.0198322	test: 314.9758612	best: 312.7655298 (8)	total: 153ms	remaining: 15.2s
10:	learn: 1036.2887450	test: 318.8695148	

## Preparing submission

In [10]:
def prepare_submission(predictions, X_test):
    index_df = X_test.index.to_frame()
    out_pd = pd.concat([index_df.reset_index(drop=True), predictions.reset_index(drop=True)], axis=1)
    out_pd = out_pd.rename(columns={0: 'prediction', 'date_forecast': 'time'})
    print(X_test.info())
    print(out_pd.info())
    out_pd['location'] = X_test['location'].reset_index(drop=True)
    out_pd.set_index('time', inplace=True)
    return out_pd

def merge_with_sample(out_pd):
    test = pd.read_csv('data/test.csv')
    test.time = pd.to_datetime(test.time)
    sample_submission = pd.read_csv('data/sample_submission.csv')
    test.set_index('time', inplace=True)
    
    merged_df = test.reset_index().merge(out_pd.reset_index(), on=['time', 'location'], how='left', suffixes=('_original', '_new'))
    merged_df['prediction_new'] = merged_df['prediction_new'].combine_first(merged_df['prediction_original'])
    merged_df.drop('prediction_original', axis=1, inplace=True)
    merged_df.rename(columns={'prediction_new': 'prediction'}, inplace=True)
    return sample_submission[['id']].merge(merged_df[['id', 'prediction']], on='id', how='left')


out_pd = prepare_submission(predictions, X_test)
sample_submission = merge_with_sample(out_pd)
sample_submission.to_csv('submission.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8640 entries, 2023-05-01 00:00:00 to 2023-07-03 23:45:00
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   absolute_humidity_2m:gm3        8640 non-null   float32
 1   air_density_2m:kgm3             8640 non-null   float32
 2   ceiling_height_agl:m            6243 non-null   float32
 3   clear_sky_energy_1h:J           8640 non-null   float32
 4   clear_sky_rad:W                 8640 non-null   float32
 5   cloud_base_agl:m                7690 non-null   float32
 6   dew_point_2m:K                  8640 non-null   float32
 7   diffuse_rad:W                   8640 non-null   float32
 8   diffuse_rad_1h:J                8640 non-null   float32
 9   direct_rad:W                    8640 non-null   float32
 10  direct_rad_1h:J                 8640 non-null   float32
 11  effective_cloud_cover:p         8640 non-null   float32
 12